## DAMAGE TOLERANCE AND THREAD ASSESMENT

Let's consider a Barely Visible Impact Damage (BVID) on a composite skin panel with integrated stringers (Category 1, CSET 4.3 Addressing Damage and Defects, slides 58-61).
The process for damage tolerance evaluation would be (AC 20-107B, Paragrah 8. Proof of Structure -  Fatigue and Damage Tolerance):

Damage assessment  and Inspection (slides 31-32, 36, 124):
   - Impact testing at various energies to establish BVID threshold.
   - Internal damage extend mapping (NDIs)
   - Microscopy to understand damage morphology.

Analysis (slide 60):
   - FEM of damaged region.
   - Prediction of residual strength.
   - Assessment of damage growth potential.
   - Must withstand Ultimate Load capability
   - Maintain structural integrity through aircraft's lifecycle.

Testing approach (slide 9, CMH-17 Building Block Testing Approach):
   - Static strength tests of damaged specimens.
   - Fatigue testing under spectrum loading.
   - Full-scale validation testing.
   - Envirommental effects must be considered.

Category 1 damage design drivers: 

Minimum Skin Thickness:
   - Must be sufficient to prevent penetration.
   - Should provide adequate impact resistance

Stringer Spacing (slide 113):
   - Affects panel buckling behavior with damage.
   - Influences damage containment capability.

Laminate Stacking (slide 14):
    - Material and stacking sequence affects damage distribution.
    - Optimization needed between matrix cracks, delamination, and fiber breakage.



One of the important aspects of the program you outlined is determining the residual strength with statistical significance so a B-Basis, let's say, in the presence of BVID can be established.  And this B-Basis must account for environmental effects.  If we have three main environments of interest (in addition to RTA), which are Cold Temperature Dry, Room Temperature Wet and Elevated Temperature Wet, do you need to do enough tests at each environment to get the scatter and the B-Basis for each and, if yes, how many tests would you recommend?

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.optimize import fsolve
from scipy.stats import anderson_ksamp
from scipy import stats
from matplotlib.patches import Patch


##### Approach 1: Full B-Basis Calculation

In [ ]:
# Approach 1: Full B-Basis Calculation
def calculate_full_b_basis_sample_size(sigma=.06, d_error=.05):
    """
    Calculation of the required sample size for B-basis values (90% probability with 95% confidence)
    Simulation values:
    Z_alpha for 90% probability = 1.645
    Z_beta for 95% confidence = 1.96
    Parameters:
    sigma = coefficient of variation
    d_error =  acceptable error
    Returns:
    n = sample size

    """
    Z_alpha = 1.645  
    Z_beta = 1.96   
    n = ((Z_alpha + Z_beta) * sigma/d_error)**2
    return np.ceil(n)

##### Approach 2: Environmental Knockdown Factors

In [ ]:
def calculate_knockdown_sample_size(std_dev=.06, k_error=.04):
    """
    Calculation of the sample size for environmental knockdown factors
    Simulation value:
    t-value for 95% confidence = 2.201
    Parameters:
    std_dev = standard deviation
    k_error = margin of error
    Returns:
    n = sample size
    """
    t_value = 2.201  
    n = (t_value * std_dev/k_error)**2
    return np.ceil(n)

##### Approach 3: Statistical Power Analysis

In [ ]:
def calculate_validation_sample_size(effect_size=1.0, power=0.8):
    """
    Calculation of the sample size for validation testing based on power analysis
    Simulation value:
    Z_alpha for 90% probability = 1.645
    Parameters:
    effect size
    power
    Returns:
    n = sample size
    """
    Z_alpha = 1.645
    Z_beta = stats.norm.ppf(power)
    n = ((Z_alpha + Z_beta)/effect_size)**2
    return np.ceil(n)

##### Function to simulate and analyze composite test data

In [11]:
def analyze_composite_data(n_samples, mean_strength=100, cv=0.06):
    """
    Simulates and analyzes composite test data
    Parameters:
    n_samples = sample size
    mean_strength
    cv
    Returns:
    data = simulation of test data (normal distribution)
    mean = mean value of the simulated data
    std =  std deviation of the simulated data
    b_basis = B-basis value
    ad_stat = the Anderson-Darling test statistic.
    ad_crit =the critical values for this distribution.
    ad_sig = the significance levels for the corresponding critical values in percents

    Anderson-Darling test statistics: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html

    """
    std_dev = mean_strength * cv
    data = np.random.normal(mean_strength, std_dev, n_samples)
    
    # Mean and standard deviation of the simulated data
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    
    # B-basis value at 90% probability
    k_value = stats.norm.ppf(0.90) 
    b_basis = mean - k_value * std

     # Anderson-Darling test for normality
    ad_stat, ad_crit, ad_sig = stats.anderson(data)
    
    return data, mean, std, b_basis, ad_stat, ad_crit, ad_sig

##### k-sample Anderson-Darling test

In [31]:
def perform_anderson_ksamp_test(samples_dict):
    """
    Perform k-sample Anderson-Darling test on multiple environmental conditions
    
    Parameters:
    samples_dict: dictionary with environmental conditions as keys and data arrays as values
    
    Returns:
    statistic: Anderson-Darling test statistic
    critical_values: critical values for different significance levels
    p_value: p-value of the test
    
    Anderson K-sample: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson_ksamp.html
    """
    # Extract samples from dictionary
    samples = list(samples_dict.values())
    
    # Perform k-sample Anderson-Darling test
    method = stats.PermutationMethod(n_resamples=9999, random_state=42)
    statistic, critical_values, p_value = anderson_ksamp(samples, method=method)

    # For composite material certification and structural analysis, we want 0.05 (5%) as the standard threshold fir critical values
    critical_value5 = critical_values[2]
    
    return statistic, critical_value5, p_value

#### Data visualization functions

In [18]:
def plot_environmental_distributions(results_dict):
    """
    Create violin plot comparing distributions across environmental conditions
    """
    plt.figure(figsize=(12, 6))
    
    # Prepare data for plotting
    data_list = []
    for condition, data in results_dict.items():
        df = pd.DataFrame({
            'Strength': data['data'],
            'Condition': condition
        })
        data_list.append(df)
    
    plot_data = pd.concat(data_list)
    
    # Create violin plot
    sns.violinplot(x='Condition', y='Strength', data=plot_data)
    plt.title('Distribution Comparison Across Environmental Conditions')
    plt.grid(True)
    plt.show()

#### Environmental conditions

In [19]:
conditions = {
    'RTD': {'mean': 100, 'cv': 0.06},
    'CTD': {'mean': 105, 'cv': 0.06},
    'RTW': {'mean': 95, 'cv': 0.06},
    'ETW': {'mean': 90, 'cv': 0.06}
}

In [20]:
# Dictionary to store results
results = {}
samples_for_ksamp = {}

# Analyze each condition
for condition, params in conditions.items():
    if condition == 'RTD':
        n_samples = int(calculate_full_b_basis_sample_size())
    else:
        n_samples = int(calculate_knockdown_sample_size())
        
    data, mean, std, b_basis, ad_stat, ad_crit, ad_sig = analyze_composite_data(
        n_samples,
        mean_strength=params['mean'],
        cv=params['cv']
    )
    
    results[condition] = {
        'data': data,
        'mean': mean,
        'std': std,
        'b_basis': b_basis,
        'anderson_stat': ad_stat,
        'anderson_crit': ad_crit,
        'anderson_sig': ad_sig
    }
    
    samples_for_ksamp[condition] = data
    
    # Print individual condition results
    print(f"\nResults for {condition}:")
    print(f"Mean: {mean:.2f}")
    print(f"Standard Deviation: {std:.2f}")
    print(f"B-basis value: {b_basis:.2f}")
    print(f"Anderson-Darling statistic: {ad_stat:.3f}")
    print("Anderson-Darling significance levels:", ad_sig)
    print("Anderson-Darling critical values:", ad_crit)


Results for RTD:
Mean: 101.71
Standard Deviation: 7.27
B-basis value: 92.40
Anderson-Darling statistic: 0.261
Anderson-Darling significance levels: [15.  10.   5.   2.5  1. ]
Anderson-Darling critical values: [0.505 0.575 0.69  0.804 0.957]

Results for CTD:
Mean: 104.61
Standard Deviation: 5.62
B-basis value: 97.41
Anderson-Darling statistic: 0.154
Anderson-Darling significance levels: [15.  10.   5.   2.5  1. ]
Anderson-Darling critical values: [0.498 0.567 0.68  0.793 0.944]

Results for RTW:
Mean: 99.72
Standard Deviation: 5.53
B-basis value: 92.63
Anderson-Darling statistic: 0.295
Anderson-Darling significance levels: [15.  10.   5.   2.5  1. ]
Anderson-Darling critical values: [0.498 0.567 0.68  0.793 0.944]

Results for ETW:
Mean: 89.79
Standard Deviation: 7.13
B-basis value: 80.66
Anderson-Darling statistic: 0.362
Anderson-Darling significance levels: [15.  10.   5.   2.5  1. ]
Anderson-Darling critical values: [0.498 0.567 0.68  0.793 0.944]


In [33]:
# Perform k-sample Anderson-Darling test
ksamp_stat, ksamp_crit, ksamp_pvalue = perform_anderson_ksamp_test(samples_for_ksamp)

print("\nK-sample Anderson-Darling test results:")
print(f"Statistic: {ksamp_stat:.3f}")
print(f"Critical value: {ksamp_crit:.3f}")
print(f"P-value: {ksamp_pvalue:.4f}")

# Interpret results
alpha = 0.05
if ksamp_pvalue < alpha:
    print("\nThe distributions are significantly different (reject null hypothesis)")
else:
    print("\nFailed to reject the null hypothesis - distributions may be similar")


K-sample Anderson-Darling test results:
Statistic: 6.292
Critical value: 1.916
P-value: 0.0001

The distributions are significantly different (reject null hypothesis)
